In [1]:
import numpy as np
import pandas as pd
import time
import geopandas as gpd
from shapely import geometry
from pyproj import CRS
from pyproj import Transformer


In [5]:
# Explore the original data for Bike in 2019-1
path = '../data/201902-citibike-tripdata.csv'
bike_data = pd.read_csv(path, dtype=str)
bike_data.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

In [6]:
# Discard some unrelated columns to simplify the problem
# Some other attributes may be used later
simplified_bike_data = bike_data.drop(['tripduration', 'start station id',
       'start station name', 'end station id', 'end station name',
       'bikeid', 'usertype', 'birth year', 'gender'], axis=1)

simplified_bike_data

,starttime,stoptime,start station latitude,start station longitude,end station latitude,end station longitude
0,2019-02-01 00:00:06.2570,2019-02-01 00:03:46.1090,40.797911,-73.9423,40.8014866,-73.9442507
1,2019-02-01 00:00:28.0320,2019-02-01 00:02:51.7460,40.72779126,-73.98564945,40.7284186,-73.98713956
2,2019-02-01 00:01:13.9870,2019-02-01 00:06:10.7340,40.676368,-73.952918,40.678045,-73.962408
3,2019-02-01 00:01:14.1520,2019-02-01 00:09:12.7870,40.7489006,-73.97604882,40.75640548,-73.9900262
4,2019-02-01 00:01:49.3410,2019-02-01 00:05:34.4980,40.76309387270797,-73.9783501625061,40.76132983124814,-73.97982001304626
...,...,...,...,...,...,...
943739,2019-02-28 23:59:43.3860,2019-03-01 00:16:02.7620,40.716887,-73.963198,40.72307749068673,-73.98583620786667
943740,2019-02-28 23:59:48.2220,2019-03-01 00:12:47.5150,40.72743423,-73.99379025,40.751873,-73.977706
943741,2019-02-28 23:59:48.6480,2019-03-01 00:11:28.5880,40.74734825,-73.99723551,40.73314259,-73.97573881
943742,2019-02-28 23:59:59.2640,2019-03-01 00:04:43.6880,40.741021509002664,-74.001384973526,40.7505853470215,-73.9946848154068


In [7]:
import csv

crs_WGS84 = CRS.from_epsg(4326)
crs_taxi_zones = CRS.from_epsg(2263)
transformer = Transformer.from_crs(crs_WGS84, crs_taxi_zones)
manhattan_zones_file = open('/Users/mhy/Code/pythonworkspace/Deep-NYC-Bike-Taxi/data-NYCZones/zones/manhattan_zones.csv', encoding='utf-8')
manhattan_zones_reader = csv.reader(manhattan_zones_file)
taxi_zones = gpd.read_file("/Users/mhy/Code/pythonworkspace/Deep-NYC-Bike-Taxi/data-NYCZones/zones/taxi_zones.shp")
manhattan_zones_numbers = []
start = 0

for i in manhattan_zones_reader:
    if start == 0:
        start = 1
        continue
    manhattan_zones_numbers.append(i[0])

manhattan_zones_numbers = manhattan_zones_numbers[1:]
manhattan_zones_numbers = [int(i) for i in manhattan_zones_numbers]
manhattan_zones = taxi_zones[taxi_zones['LocationID'].isin(manhattan_zones_numbers)]
manhattan_zones

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mhy/Code/pythonworkspace/Deep-NYC-Bike-Taxi/data-NYCZones/zones/manhattan_zones.csv'

In [29]:
simplified_bike_data = simplified_bike_data.head(1000)

In [30]:
def get_PULocationID(row):
    start_latitude = row['start station latitude']
    start_longitude = row['start station longitude']
    x, y = transformer.transform(start_latitude, start_longitude)
    for index, row in manhattan_zones.iterrows():
        if geometry.Point(x, y).within(row['geometry']):
            return index
        
def get_DOLocationID(row):
    stop_latitude = row['end station latitude']
    stop_longitude = row['end station longitude']
    x, y = transformer.transform(stop_latitude, stop_longitude)
    for index, row in manhattan_zones.iterrows():
        if geometry.Point(x, y).within(row['geometry']):
            return index
        
simplified_bike_data['PULocationID'] = simplified_bike_data.apply(get_PULocationID, axis=1)
simplified_bike_data['DOLocationID'] = simplified_bike_data.apply(get_DOLocationID, axis=1)
simplified_bike_data

/var/folders/n3/ddy0x11x3b54hh3tcq3sqdxc0000gn/T/ipykernel_4577/3533701381.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplified_bike_data['PULocationID'] = simplified_bike_data.apply(get_PULocationID, axis=1)
/var/folders/n3/ddy0x11x3b54hh3tcq3sqdxc0000gn/T/ipykernel_4577/3533701381.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplified_bike_data['DOLocationID'] = simplified_bike_data.apply(get_DOLocationID, axis=1)


,starttime,stoptime,start station latitude,start station longitude,end station latitude,end station longitude,PULocationID,DOLocationID
0,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,40.77896784,-73.97374737,40.7882213,-73.97041561,42.0,237.0
1,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,40.751873,-73.977706,40.74780373,-73.9734419,169.0,169.0
2,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,40.78524672,-73.97667321,40.77314236,-73.95856158,238.0,235.0
3,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,40.73221853,-73.98165557,40.738046142482766,-73.99642959237099,223.0,89.0
4,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,40.72743423,-73.99379025,40.73827428,-73.98751968,113.0,106.0
...,...,...,...,...,...,...,...,...
995,2019-01-01 05:01:23.5070,2019-01-01 05:07:03.7980,40.71273266,-74.0046073,40.71870987,-74.0090009,230.0,230.0
996,2019-01-01 05:01:13.1600,2019-01-01 05:37:06.2740,40.75660359,-73.9979009,40.74311555376486,-73.98215353488922,245.0,169.0
997,2019-01-01 05:02:19.1090,2019-01-01 05:05:55.8140,40.805726,-73.936322,40.8107922,-73.9430681,73.0,41.0
998,2019-01-01 05:02:32.3370,2019-01-01 05:09:17.6580,40.77579376683666,-73.9762057363987,40.7689738,-73.95482273,42.0,139.0


In [31]:
simplified_bike_data = simplified_bike_data.dropna(how='any')
simplified_bike_data

,starttime,stoptime,start station latitude,start station longitude,end station latitude,end station longitude,PULocationID,DOLocationID
0,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,40.77896784,-73.97374737,40.7882213,-73.97041561,42.0,237.0
1,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,40.751873,-73.977706,40.74780373,-73.9734419,169.0,169.0
2,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,40.78524672,-73.97667321,40.77314236,-73.95856158,238.0,235.0
3,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,40.73221853,-73.98165557,40.738046142482766,-73.99642959237099,223.0,89.0
4,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,40.72743423,-73.99379025,40.73827428,-73.98751968,113.0,106.0
...,...,...,...,...,...,...,...,...
993,2019-01-01 04:59:35.2800,2019-01-01 05:06:06.9520,40.75019995,-73.99093085,40.75500254,-73.98014437,185.0,160.0
995,2019-01-01 05:01:23.5070,2019-01-01 05:07:03.7980,40.71273266,-74.0046073,40.71870987,-74.0090009,230.0,230.0
996,2019-01-01 05:01:13.1600,2019-01-01 05:37:06.2740,40.75660359,-73.9979009,40.74311555376486,-73.98215353488922,245.0,169.0
997,2019-01-01 05:02:19.1090,2019-01-01 05:05:55.8140,40.805726,-73.936322,40.8107922,-73.9430681,73.0,41.0


In [32]:
simplified_bike_data['DOLocationID'] = simplified_bike_data['DOLocationID'].astype('int64')
simplified_bike_data['PULocationID'] = simplified_bike_data['PULocationID'].astype('int64')
simplified_bike_data['starttime'] = pd.to_datetime(simplified_bike_data['starttime'])
simplified_bike_data['stoptime'] = pd.to_datetime(simplified_bike_data['stoptime'])
simplified_bike_data

/var/folders/n3/ddy0x11x3b54hh3tcq3sqdxc0000gn/T/ipykernel_4577/1609778386.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplified_bike_data['DOLocationID'] = simplified_bike_data['DOLocationID'].astype('int64')
/var/folders/n3/ddy0x11x3b54hh3tcq3sqdxc0000gn/T/ipykernel_4577/1609778386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplified_bike_data['PULocationID'] = simplified_bike_data['PULocationID'].astype('int64')
/var/folders/n3/ddy0x11x3b54hh3tcq3sqdxc0000gn/T/ipykernel_4577/1609778386

,starttime,stoptime,start station latitude,start station longitude,end station latitude,end station longitude,PULocationID,DOLocationID
0,2019-01-01 00:01:47.401,2019-01-01 00:07:07.581,40.77896784,-73.97374737,40.7882213,-73.97041561,42,237
1,2019-01-01 00:04:43.736,2019-01-01 00:10:00.608,40.751873,-73.977706,40.74780373,-73.9734419,169,169
2,2019-01-01 00:06:03.997,2019-01-01 00:15:55.438,40.78524672,-73.97667321,40.77314236,-73.95856158,238,235
3,2019-01-01 00:07:03.545,2019-01-01 00:52:22.650,40.73221853,-73.98165557,40.738046142482766,-73.99642959237099,223,89
4,2019-01-01 00:07:35.945,2019-01-01 00:12:39.502,40.72743423,-73.99379025,40.73827428,-73.98751968,113,106
...,...,...,...,...,...,...,...,...
993,2019-01-01 04:59:35.280,2019-01-01 05:06:06.952,40.75019995,-73.99093085,40.75500254,-73.98014437,185,160
995,2019-01-01 05:01:23.507,2019-01-01 05:07:03.798,40.71273266,-74.0046073,40.71870987,-74.0090009,230,230
996,2019-01-01 05:01:13.160,2019-01-01 05:37:06.274,40.75660359,-73.9979009,40.74311555376486,-73.98215353488922,245,169
997,2019-01-01 05:02:19.109,2019-01-01 05:05:55.814,40.805726,-73.936322,40.8107922,-73.9430681,73,41


In [12]:
simplified_bike_data.columns

Index(['starttime', 'stoptime', 'start station latitude',
       'start station longitude', 'end station latitude',
       'end station longitude', 'PULocationID', 'DOLocationID'],
      dtype='object')

In [33]:
simplified_bike_data['starttime'] = simplified_bike_data['starttime'].astype(str)
simplified_bike_data['stoptime'] = simplified_bike_data['stoptime'].astype(str)

bike_inflow = simplified_bike_data.drop(['starttime', 'PULocationID', 'start station latitude','start station longitude', 'end station latitude','end station longitude'], axis=1)
bike_outflow = simplified_bike_data.drop(['stoptime', 'DOLocationID', 'start station latitude','start station longitude', 'end station latitude','end station longitude'], axis=1)

/var/folders/n3/ddy0x11x3b54hh3tcq3sqdxc0000gn/T/ipykernel_4577/2258161701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplified_bike_data['starttime'] = simplified_bike_data['starttime'].astype(str)
/var/folders/n3/ddy0x11x3b54hh3tcq3sqdxc0000gn/T/ipykernel_4577/2258161701.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplified_bike_data['stoptime'] = simplified_bike_data['stoptime'].astype(str)


In [34]:
bike_outflow

,starttime,PULocationID
0,2019-01-01 00:01:47.401,42
1,2019-01-01 00:04:43.736,169
2,2019-01-01 00:06:03.997,238
3,2019-01-01 00:07:03.545,223
4,2019-01-01 00:07:35.945,113
...,...,...
993,2019-01-01 04:59:35.280,185
995,2019-01-01 05:01:23.507,230
996,2019-01-01 05:01:13.160,245
997,2019-01-01 05:02:19.109,73


In [38]:
from kafka import KafkaProducer

# Export data to kafka
producer = KafkaProducer(bootstrap_servers='localhost:9092')

# Write 100 messages to kafka
# The number need to be considered later
N = 635
for i in range(N):
    json_string = simplified_bike_data.iloc[i].to_json()
    inflow_string = bike_inflow.iloc[i].to_json()
    outflow_string = bike_outflow.iloc[i].to_json()
    # To send a message
    producer.send('bike_origin', json_string.encode('utf-8'))
    producer.send('bike_inflow', inflow_string.encode('utf-8'))
    producer.send('bike_outflow', outflow_string.encode('utf-8'))
    time.sleep(0.1)

producer.flush()  # Wait for any outstanding messages to be transmitted and delivery acknowledgments received
producer.close()

In [22]:
from sqlalchemy import create_engine

USERNAME = 'root'
PASSWORD = 'MySQLroot'
HOST = 'localhost'
DATABASE_NAME = 'traffic'

engine = create_engine(f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{HOST}/{DATABASE_NAME}')

# Create a table in MySQL and insert the data
# Convert a dataframe to SQL create table statement
manhattan_bike_sampled = simplified_bike_data[:100]
table_name = 'manhattan_bike'
manhattan_bike_sampled.to_sql(table_name, engine, if_exists='replace', index=False)
create_sql = pd.io.sql.get_schema(manhattan_bike_sampled, table_name, con=engine)
print(create_sql)


CREATE TABLE manhattan_bike (
	starttime TEXT, 
	stoptime TEXT, 
	`start station latitude` TEXT, 
	`start station longitude` TEXT, 
	`end station latitude` TEXT, 
	`end station longitude` TEXT, 
	`PULocationID` BIGINT, 
	`DOLocationID` BIGINT
)




In [ ]:
CREATE TABLE manhattan_bike (
	starttime TIMESTAMP, 
	stoptime TIMESTAMP, 
	`start station latitude` FLOAT, 
	`start station longitude` FLOAT, 
	`end station latitude` FLOAT, 
	`end station longitude` FLOAT, 
	`PULocationID` BIGINT, 
	`DOLocationID` BIGINT
) WITH (
    'connector' = 'kafka',  -- using kafka connector
    'topic' = 'bike_origin',  -- kafka topic
    'scan.startup.mode' = 'earliest-offset',  -- reading from the beginning
    'properties.bootstrap.servers' = 'localhost:9092',  -- kafka broker address
    'format' = 'json'  -- the data format is json
);

CREATE TABLE bike_inflow (
	stoptime TIMESTAMP(3), 
	`DOLocationID` BIGINT, 
    WATERMARK FOR stoptime AS stoptime - INTERVAL '5' SECOND
)WITH (
    'connector' = 'kafka',  -- using kafka connector
    'topic' = 'bike_inflow',  -- kafka topic
    'scan.startup.mode' = 'earliest-offset',  -- reading from the beginning
    'properties.bootstrap.servers' = 'localhost:9092',  -- kafka broker address
    'format' = 'json'  -- the data format is json
);

CREATE TABLE bike_outflow (
	starttime TIMESTAMP(3), 
	`PULocationID` BIGINT, 
    WATERMARK FOR starttime AS starttime - INTERVAL '5' SECOND
)WITH (
    'connector' = 'kafka',  -- using kafka connector
    'topic' = 'bike_outflow',  -- kafka topic
    'scan.startup.mode' = 'earliest-offset',  -- reading from the beginning
    'properties.bootstrap.servers' = 'localhost:9092',  -- kafka broker address
    'format' = 'json'  -- the data format is json
);


In [ ]:
CREATE TABLE bike_origin_es (
    starttime TIMESTAMP, 
	stoptime TIMESTAMP, 
	`start station latitude` FLOAT, 
	`start station longitude` FLOAT, 
	`end station latitude` FLOAT, 
	`end station longitude` FLOAT, 
	`PULocationID` BIGINT, 
	`DOLocationID` BIGINT
) WITH (
    'connector' = 'elasticsearch-7', -- using elasticsearch connector
    'hosts' = 'https://demo0.es.asia-southeast1.gcp.elastic-cloud.com:9243',  -- elasticsearch address
    'username' = 'elastic',
    'password' = 'G1Tqo3onjeIg7G8E6zSIWlq8',
    'index' = 'bike_origin_es'  -- elasticsearch index name, similar to database table name
);

INSERT INTO bike_origin_es (SELECT * FROM manhattan_bike);

In [21]:
# Read data from elasticsearch
from datetime import datetime
from elasticsearch import Elasticsearch

es = Elasticsearch(hosts='http://localhost:9200')

indices = es.indices.get_alias(index="*")
indices

/var/folders/n3/ddy0x11x3b54hh3tcq3sqdxc0000gn/T/ipykernel_21526/1882461776.py:7: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  indices = es.indices.get_alias(index="*")
/var/folders/n3/ddy0x11x3b54hh3tcq3sqdxc0000gn/T/ipykernel_21526/1882461776.py:7: ElasticsearchWarning: this request accesses system indices: [.kibana_7.17.9_001, .tasks, .async-search, .kibana_task_manager_7.17.9_001, .apm-agent-configuration, .apm-custom-link], but in a future major version, direct access to system indices will be prevented by default
  indices = es.indices.get_alias(index="*")


ObjectApiResponse({'.kibana-event-log-7.17.9-000001': {'aliases': {'.kibana-event-log-7.17.9': {'is_write_index': True, 'is_hidden': True}}}, '.kibana_7.17.9_001': {'aliases': {'.kibana': {}, '.kibana_7.17.9': {}}}, '.tasks': {'aliases': {}}, 'manhattan_zones': {'aliases': {}}, 'taxi_inflow_es': {'aliases': {'joined': {}}}, '.async-search': {'aliases': {}}, '.kibana_task_manager_7.17.9_001': {'aliases': {'.kibana_task_manager': {}, '.kibana_task_manager_7.17.9': {}}}, 'reprojected_manhattan_data': {'aliases': {'joined': {}}}, '.apm-agent-configuration': {'aliases': {}}, '.apm-custom-link': {'aliases': {}}})

In [34]:
res = es.get(index="joined", id='9KYoIocBz7EFPZLESiTq')
res['_source']

/var/folders/n3/ddy0x11x3b54hh3tcq3sqdxc0000gn/T/ipykernel_21526/2017513511.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.get(index="joined", id='9KYoIocBz7EFPZLESiTq')


{'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 162, 'inflow': 1}

In [51]:
all_docs = es.search(index='joined', body={"query": {"match_all": {}}}, size=2000)
# print the results
docs = [doc['_source'] for doc in all_docs['hits']['hits']]
docs

/var/folders/n3/ddy0x11x3b54hh3tcq3sqdxc0000gn/T/ipykernel_21526/793078707.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  all_docs = es.search(index='joined', body={"query": {"match_all": {}}}, size=2000)
/var/folders/n3/ddy0x11x3b54hh3tcq3sqdxc0000gn/T/ipykernel_21526/793078707.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  all_docs = es.search(index='joined', body={"query": {"match_all": {}}}, size=2000)


[{'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 162, 'inflow': 1},
 {'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 234, 'inflow': 1},
 {'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 237, 'inflow': 1},
 {'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 41, 'inflow': 1},
 {'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 74, 'inflow': 1},
 {'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 75, 'inflow': 1},
 {'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 68, 'inflow': 3},
 {'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 234, 'inflow': 2},
 {'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 100, 'inflow': 1},
 {'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 107, 'inflow': 1},
 {'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 158, 'inflow': 2},
 {'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 239, 'inflow': 2},
 {'hour_start': '2019-01-01 00:00:00', 'DOLocationID': 68, 'inflow': 4},
 {'hour_start': '2019-01-01 00:00:00', 'DOL

In [55]:
from itertools import groupby
from operator import itemgetter

# Assuming your list of dictionaries is called `data`
# Sort the list by hour_start and LocationID
data = sorted(docs, key=itemgetter('hour_start', 'DOLocationID'))

# Define a key function that extracts the hour_start and LocationID values
keyfunc = itemgetter('hour_start', 'DOLocationID')

# Use itertools.groupby() to group the data by hour_start and LocationID
groups = groupby(data, key=keyfunc)

# Calculate the max inflow for each group and store it in a list
max_inflows = []
for (hour_start, location_id), group in groups:
    max_inflow = max(group, key=itemgetter('inflow'))['inflow']
    max_inflows.append({'hour_start': hour_start, 'LocationID': location_id, 'max_inflow': max_inflow})

# Print the results
max_inflows

[{'hour_start': '2019-01-01 00:00:00', 'LocationID': 4, 'max_inflow': 7},
 {'hour_start': '2019-01-01 00:00:00', 'LocationID': 12, 'max_inflow': 1},
 {'hour_start': '2019-01-01 00:00:00', 'LocationID': 13, 'max_inflow': 6},
 {'hour_start': '2019-01-01 00:00:00', 'LocationID': 24, 'max_inflow': 5},
 {'hour_start': '2019-01-01 00:00:00', 'LocationID': 41, 'max_inflow': 10},
 {'hour_start': '2019-01-01 00:00:00', 'LocationID': 42, 'max_inflow': 11},
 {'hour_start': '2019-01-01 00:00:00', 'LocationID': 43, 'max_inflow': 9},
 {'hour_start': '2019-01-01 00:00:00', 'LocationID': 45, 'max_inflow': 1},
 {'hour_start': '2019-01-01 00:00:00', 'LocationID': 48, 'max_inflow': 12},
 {'hour_start': '2019-01-01 00:00:00', 'LocationID': 50, 'max_inflow': 11},
 {'hour_start': '2019-01-01 00:00:00', 'LocationID': 68, 'max_inflow': 44},
 {'hour_start': '2019-01-01 00:00:00', 'LocationID': 74, 'max_inflow': 11},
 {'hour_start': '2019-01-01 00:00:00', 'LocationID': 75, 'max_inflow': 11},
 {'hour_start': '20

In [59]:
import numpy as np

# Assuming your grouped list is called `max_inflows`
# Create a numpy array with 24 rows (for 24 hour_start values) and 69 columns (for 69 LocationID values)
hour_starts = sorted(set(item['hour_start'] for item in max_inflows))
locations = sorted(set(item['LocationID'] for item in max_inflows))
max_inflows_array = np.zeros((len(hour_starts), 69))

# Fill in the array with the max inflow values, using the hour_start and LocationID as indices
for item in max_inflows:
    i = hour_starts.index(item['hour_start'])
    j = locations.index(item['LocationID'])
    max_inflows_array[i, j] = item['max_inflow']

# Print the resulting array
print(max_inflows_array)


[[ 7.  1.  6.  5. 10. 11.  9.  1. 12. 11. 44. 11. 11. 43. 11.  5. 21.  5.
  30. 17. 14.  4.  1. 20. 24. 40. 21. 14. 13. 14. 20. 12. 19. 29.  6. 11.
   6. 41. 11.  3.  8. 15. 43. 13. 10. 27. 29. 50. 39. 24. 29.  3.  4. 23.
  30.  2. 21. 36.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
